In [18]:
import pandas as pd
import numpy as np

In [6]:
domain_mx = pd.read_csv("data/gov_mx_results.csv").drop_duplicates()
mx_ip = pd.read_csv("data/gov_mailserver_a_results.csv").drop_duplicates()
ip_geodata = pd.read_csv("data/gov_mailserver_geo.csv").drop_duplicates()

In [2]:
domain_mx = pd.read_csv("data/gov_mx_results.csv")
mx_ip = pd.read_csv("data/gov_mailserver_a_results.csv")
ip_geodata = pd.read_csv("data/gov_mailserver_geo.csv")

In [5]:
print(domain_mx.shape, domain_mx.drop_duplicates().shape)
print(mx_ip.shape, mx_ip.drop_duplicates().shape)
print(ip_geodata.shape, ip_geodata.drop_duplicates().shape)

mx_ip.drop_duplicates().to_csv('data/gov_mailserver_a_results_clean.csv', index=False)
ip_geodata.drop_duplicates().to_csv('data/gov_mailserver_geo_clean.csv', index=False)

(13879, 5) (13879, 5)
(33869, 2) (23754, 2)
(33867, 10) (2281, 10)


In [28]:
data_mx_ip = pd.merge(domain_mx, mx_ip, how="left", left_on="exchange", right_on="url")
data_mx_ip = data_mx_ip.drop(columns=["url_y"]).rename(columns={"url_x": "domain"})
data_ip_geo = pd.merge(mx_ip, ip_geodata, how="left", left_on="ipv4", right_on="ip")
data_ip_geo = data_ip_geo.drop(columns=["ip"])

In [4]:
gov_domains = domain_mx.url.unique()
mailserver_domains = domain_mx.exchange.unique()
mailserver_ips = mx_ip.ipv4.unique()
mailserver_coords = ip_geodata[["country_name", "latitude", "longitude"]].drop_duplicates()

print("Unique .gov domains: ", len(gov_domains))
print("Unique Mailserver domains: ", len(mailserver_domains))
print("Unique Mailserver IPs: ", len(mailserver_ips))
print("Unique Mailserver locations: ", mailserver_coords.shape[0])
print("Locations in US: ", mailserver_coords[mailserver_coords["country_name"] == "United States"].shape[0])

Unique .gov domains:  7391
Unique Mailserver domains:  7471
Unique Mailserver IPs:  2282
Unique Mailserver locations:  524
Locations in US:  504


In [5]:
try:  
    mailserver_coords= mailserver_coords.reset_index()
except:
    print("already reset!")

locations = []
location_to_ip = dict()
for i in range(0, mailserver_coords.shape[0]):
    loc = mailserver_coords.loc[i]
    ips = ip_geodata[ip_geodata["latitude"] == loc["latitude"]]
    ips = ips[ips["longitude"] == loc["longitude"]]
    coord = loc["latitude"], loc["longitude"]
    locations.append(coord)
    location_to_ip[coord] = ips

In [6]:
loc_ip_data_summ = []
for location in locations:
    num = len(location_to_ip[location])
    loc_ip_data_summ.append([location, num])

loc_ip_data_summ_df = pd.DataFrame(loc_ip_data_summ, columns=['Location', '# IPs'])

In [36]:
valid_locations = loc_ip_data_summ_df.sort_values(by=['# IPs'], ascending=False).drop(4).drop(152).reset_index().drop(columns=["index"])

In [29]:
print("Total IPs: ", valid_locations.shape[0])
print("Number of locations with greater than 5 IPs: ", valid_locations[valid_locations["# IPs"] > 5].shape[0])
print("Number of locations with 5 IPs: ", valid_locations[valid_locations["# IPs"] == 5].shape[0])
print("Number of locations with 4 IPs: ", valid_locations[valid_locations["# IPs"] == 4].shape[0])
print("Number of locations with 3 IPs: ", valid_locations[valid_locations["# IPs"] == 3].shape[0])
print("Number of locations with 2 IPs: ", valid_locations[valid_locations["# IPs"] == 2].shape[0])
print("Number of locations with 1 IP: ", valid_locations[valid_locations["# IPs"] == 1].shape[0])

Total IPs:  522
Number of locations with greater than 5 IPs:  22
Number of locations with 5 IPs:  4
Number of locations with 4 IPs:  6
Number of locations with 3 IPs:  31
Number of locations with 2 IPs:  100
Number of locations with 1 IP:  359


In [29]:
data_ip_geo.head(5)

,url,ipv4,city_name,country_name,country_iso_code,most_specific_name,most_specific_iso_code,latitude,longitude,postal,network
0,59dcmi-gov.mail.protection.outlook.com.,52.101.9.5,Washington,United States,US,Virginia,VA,38.7095,-78.1539,22747,52.101.9.0/24
1,59dcmi-gov.mail.protection.outlook.com.,52.101.42.13,NaN,United States,US,Washington,WA,47.6034,-122.3414,NaN,52.101.42.0/24
2,59dcmi-gov.mail.protection.outlook.com.,52.101.41.4,San Jose,United States,US,California,CA,37.1835,-121.7714,95141,52.101.41.0/24
3,59dcmi-gov.mail.protection.outlook.com.,52.101.9.2,Washington,United States,US,Virginia,VA,38.7095,-78.1539,22747,52.101.9.0/24
4,59dcmi-gov.mail.protection.outlook.com.,52.101.9.11,Washington,United States,US,Virginia,VA,38.7095,-78.1539,22747,52.101.9.0/24


In [31]:
url_to_loc = []
for url in domain_mx["url"].unique():
    mx_domains = domain_mx[domain_mx["url"] == url]["exchange"].to_numpy()
    #mx_ips = mx_ip[mx_ip["url"].isin(mx_domains)]["ipv4"].to_numpy()
    count = data_ip_geo[data_ip_geo["url"].isin(mx_domains)].shape[0]
    url_to_loc.append([url, count])

In [33]:
url_to_loc_df = pd.DataFrame(url_to_loc, columns=['url', '# Locations'])

In [ ]:
url_to_loc_df.to_csv('data/domain_location_count.csv', index=False)